In [3]:
import pygame, sys, random
pygame.mixer.pre_init(frequency=44100, size=-16, channels=1, buffer=512)
pygame.init()
pygame.display.set_caption("Flapy Bird")
screen= pygame.display.set_mode((432,768))
gamefont = pygame.font.Font('04B_19.TTF',30)
FPSCLOCK=pygame.time.Clock()
#music=pygame.mixer.music.load("aaa.mp3")
#pygame.mixer.music.play()

#Các biến trò chơi
trongluc=0.3
chimchuyendong= 0
game_active=True
score = 0
high_score= 0
#hình nền
bg = pygame.transform.scale2x(pygame.image.load("background.png")).convert()

#Tạo Sàn 
san = pygame.transform.scale2x(pygame.image.load("san.png")).convert()
floor_x_pos = 0

# Tạo Chim
bird_down = (pygame.image.load("yellowbird-downflap.png").convert_alpha())
bird_mid = (pygame.image.load("yellowbird-midflap.png").convert_alpha())
bird_up = (pygame.image.load("yellowbird-upflap.png").convert_alpha())
bird_list= [bird_down,bird_mid,bird_up] 
bird_index = 0
bird = bird_list[bird_index]
bird_rect = bird.get_rect(center = (100,384))
#Thời gian chim đập cánh
chimdapcanh= pygame.USEREVENT + 1
pygame.time.set_timer(chimdapcanh,200)

#Tạo ống
ong = pygame.image.load("ong.png").convert_alpha()
ong = pygame.transform.scale2x(ong)
list_ong=[]
# thời gian tạo ống
hienong= pygame.USEREVENT
pygame.time.set_timer(hienong, 1300)
chieudaiong = [200,300,400]
# Tạo nền kết thúc
ketthuc = pygame.image.load("message.png").convert_alpha()
ketthuc=pygame.transform.scale2x(ketthuc)
ketthuc_rect=ketthuc.get_rect(center=(216,384))
#Chèn âm thanh
flap_sound = pygame.mixer.Sound("sfx_wing.wav")
hit_sound = pygame.mixer.Sound("sfx_hit.wav")
score_sound = pygame.mixer.Sound("sfx_point.wav")
score_sound_countdown = 100

#Các hàm của trò chơi
#hàm sàn 
def floor_draw():
    screen.blit(san,(floor_x_pos,650))
    screen.blit(san,(floor_x_pos+432,650))
#hàm ống:
def pipe():
    random_pipe= random.choice(chieudaiong)
    bottompipe = ong.get_rect(midtop=(500,random_pipe))
    toppipe = ong.get_rect(midtop=(500,random_pipe-650))
    return toppipe,bottompipe
def ongdichuyen(ongs):
    for pipe in ongs:
        pipe.centerx -=5
    return ongs
def veong(ongs):
    for pipe in ongs:
        if pipe.bottom >=600:
            screen.blit(ong,pipe)
        else:
            latong= pygame.transform.flip(ong,False,True)
            screen.blit(latong,pipe)
def check_vacham(ongs):
    for pipe in ongs:
        if bird_rect.colliderect(pipe):
            hit_sound.play()                     
            return False
        if bird_rect.top <= -1 or bird_rect.bottom >=650:
            hit_sound.play()
            return False
    return True
def xoay_chim(birds):
    new_bird = pygame.transform.rotozoom(birds,-chimchuyendong*3,1)
    return new_bird
def bird_animation():
    new_bird = bird_list[bird_index]
    newbird_rect=new_bird.get_rect(center=(100,bird_rect.centery))
    return new_bird, newbird_rect
def hiendiem(game_sta):
    if game_sta == "main game":
        diem=gamefont.render(str(int(score)),True,(255,255,255))
        diem_rect=diem.get_rect(center=(216,100))
        screen.blit(diem,diem_rect)
    if game_sta == "game over":
        diem=gamefont.render(f'Score:{int(score)}',True,(255,255,255))
        diem_rect=diem.get_rect(center=(216,100))
        screen.blit(diem,diem_rect)
        
        high_socre=gamefont.render(f'High Score:{int(high_score)}',True,(255,255,255))
        high_score_rect=diem.get_rect(center=(190,620))
        screen.blit( high_socre, high_score_rect)
def update_score(score,high_score):
    if score>high_score:
        high_score=score
    return high_score

#Vòng lặp trò chơi 
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE and game_active:
                flap_sound.play()                 
                chimchuyendong = 0
                chimchuyendong = -6
            if event.key == pygame.K_SPACE and game_active==False:   
                game_active=True
                list_ong.clear()
                bird_rect.center =(100,384)
                chimchuyendong= 0
                score = 0
                #pygame.mixer.music.play()
        if event.type == hienong:
            list_ong.extend(pipe())
        if event.type == chimdapcanh:
            if bird_index < 2:
                bird_index += 1
            else:
                bird_index = 0
        bird, bird_rect = bird_animation()
                      
    #nền      
    screen.blit(bg,(0,0)) 
    if game_active:
        #chim
        chimchuyendong+=trongluc
        xoaychim=xoay_chim(bird)
        bird_rect.centery += chimchuyendong 
        screen.blit(xoaychim,bird_rect) 
        game_active= check_vacham(list_ong)
        #Ống
        list_ong = ongdichuyen(list_ong)
        veong(list_ong)
        #Điểm
        score += 0.01
        hiendiem("main game")
        score_sound_countdown -= 1
        if score_sound_countdown <= 0:
            score_sound.play()
            score_sound_countdown = 100
    else:
        hiendiem("game over")
        high_score=update_score(score,high_score)
        screen.blit(ketthuc,ketthuc_rect)
    #Sàn
    floor_x_pos -= 1
    floor_draw()
    if floor_x_pos<=-432:
        floor_x_pos =0
   
    pygame.display.update()
    FPSCLOCK.tick(60) 

SystemExit: 